# Scenario
We imagine we are the analysts for a Dota 2 team called "Team Secret". 

In [83]:
import ipywidgets as widgets
import pickle
import pandas as pd

class DotaTeam:
    def __init__(self,team_id,name):
        self.team_id = team_id
        self.name = name
    def __str__(self):
        return str(self.__dict__)
    def __repr__(self):
        return str(self)
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__,
            sort_keys=True, indent=4)

def load_obj(name):
    with open('./Data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [ ]:
# Load data from Data_Collection
data = load_obj('matchdata_19_20')

In [101]:
# Load heroes meta data
df_hero_list = pd.read_json('Data/heroes.json').T
df_hero_list = df_hero_list[['id','localized_name']]
df_hero_list.set_index("id", inplace = True)

In [30]:
teams_list = []

for match in data:
    if 'radiant_team' in match:
        if match['radiant_team']['name'] != '':
            if not any(team.team_id == match['radiant_team']['team_id'] for team in teams_list):
                teams_list.append(DotaTeam(match['radiant_team']['team_id'], match['radiant_team']['name']))
    if 'dire_team' in match:
        if match['dire_team']['name'] != '':
            if not any(team.team_id == match['dire_team']['team_id'] for team in teams_list):
                teams_list.append(DotaTeam(match['dire_team']['team_id'], match['dire_team']['name']))

In [33]:
print('Number of unique teams: ', len(teams_list))

Number of unique teams:  2011


In [110]:
# General information about own team
team_secret_matches = []
for match in data:
    if 'radiant_team' in match:
        if match['radiant_team']['team_id'] == 1838315:
            team_secret_matches.append(match)
    if 'dire_team' in match:
        if match['dire_team']['team_id'] == 1838315:
            team_secret_matches.append(match)
            
# Calculate key values
team_secret_picked_heroes = []
team_secret_banned_heroes = []
team_secret_mean_match_time = 0

for match in team_secret_matches:
    team_secret_mean_match_time += match['duration']
    # If they are or radiant
    if 'radiant_team' in match:
        if match['radiant_team']['team_id'] == 1838315 and match['picks_bans'] is not None:
            for pick_ban in match['picks_bans']:
                # Add to picked heroes
                if pick_ban['is_pick']:
                    if pick_ban['team'] == 0:
                        team_secret_picked_heroes.append(pick_ban['hero_id'])
                # Add to banned heroes
                else:
                    if pick_ban['team'] == 0:
                        team_secret_banned_heroes.append(pick_ban['hero_id'])
    # If they are or dire
    if 'dire_team' in match:
        if match['dire_team']['team_id'] == 1838315 and match['picks_bans'] is not None:
            for pick_ban in match['picks_bans']:
                # Add to picked heroes
                if pick_ban['is_pick']:
                    if pick_ban['team'] == 1:
                        team_secret_picked_heroes.append(pick_ban['hero_id'])
                # Add to banned heroes            
                else:
                    if pick_ban['team'] == 1:
                        team_secret_banned_heroes.append(pick_ban['hero_id'])

# Average game length
team_secret_mean_match_time = team_secret_mean_match_time/len(team_secret_matches)


df_secret_picked_heroes = pd.DataFrame(team_secret_picked_heroes)
secret_picks = df_secret_picked_heroes[0].value_counts()

df_secret_banned_heroes = pd.DataFrame(team_secret_banned_heroes)
secret_bans = df_secret_banned_heroes[0].value_counts()

temp = df_hero_list.join(secret_picks, how = "outer")
temp.columns = ['Hero Name', 'Number Picks']

secret_heroes = temp.join(secret_bans, how = "outer")
secret_heroes.columns = ['Hero Name', 'Number Picks', 'Number Bans']

secret_heroes.fillna(0)

,Hero Name,Number Picks,Number Bans
1,Anti-Mage,7.0,3.0
2,Axe,1.0,8.0
3,Bane,9.0,15.0
4,Bloodseeker,15.0,14.0
5,Crystal Maiden,4.0,0.0
...,...,...,...
121,Grimstroke,5.0,34.0
123,Hoodwink,0.0,0.0
126,Void Spirit,25.0,15.0
128,Snapfire,13.0,12.0


In [109]:
secret_heroes

,Hero Name,Number Picks,Number Bans
1,Anti-Mage,7.0,3.0
2,Axe,1.0,8.0
3,Bane,9.0,15.0
4,Bloodseeker,15.0,14.0
5,Crystal Maiden,4.0,NaN
...,...,...,...
121,Grimstroke,5.0,34.0
123,Hoodwink,NaN,NaN
126,Void Spirit,25.0,15.0
128,Snapfire,13.0,12.0


In [47]:
team_to_analyze_input = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)

        
display(team_to_analyze_input)

Text(value='Hello World', description='String:', placeholder='Type something')

In [76]:
team_to_analyze

1838315

In [74]:
# Analyze opponent
team_to_analyze = 0

for team in teams_list:
    if team.name == team_to_analyze_input.value:
        team_to_analyze = team.team_id
if team_to_analyze == 0:
    print('Team not found!')

# Filter opponent matches
opponent_matches = []

for match in data:
    if 'radiant_team' in match:
        if match['radiant_team']['team_id'] == team_to_analyze:
            opponent_matches.append(match)
    if 'dire_team' in match:
        if match['dire_team']['team_id'] == team_to_analyze:
            opponent_matches.append(match)
            
# Calculate key values
opponent_picked_heroes = []
opponent_banned_heroes = []
opponent_mean_match_time = 0

for match in opponent_matches:
    opponent_mean_match_time += match['duration']
    # If they are or radiant
    if 'radiant_team' in match:
        if match['radiant_team']['team_id'] == team_to_analyze and match['picks_bans'] is not None:
            for pick_ban in match['picks_bans']:
                # Add to picked heroes
                if pick_ban['is_pick']:
                    if pick_ban['team'] == 0:
                        opponent_picked_heroes.append(pick_ban['hero_id'])
                # Add to banned heroes
                else:
                    if pick_ban['team'] == 0:
                        opponent_banned_heroes.append(pick_ban['hero_id'])
    # If they are or dire
    if 'dire_team' in match:
        if match['dire_team']['team_id'] == team_to_analyze and match['picks_bans'] is not None:
            for pick_ban in match['picks_bans']:
                # Add to picked heroes
                if pick_ban['is_pick']:
                    if pick_ban['team'] == 1:
                        opponent_picked_heroes.append(pick_ban['hero_id'])
                # Add to banned heroes            
                else:
                    if pick_ban['team'] == 1:
                        opponent_banned_heroes.append(pick_ban['hero_id'])
                    
opponent_mean_match_time = opponent_mean_match_time/len(opponent_matches)